In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3

In [2]:
account = pd.read_csv('data/account.csv', delimiter=";")
card_test = pd.read_csv('data/card_test.csv', delimiter=";")
card_train = pd.read_csv('data/card_train.csv', delimiter=";")
client = pd.read_csv('data/client.csv', delimiter=";")
disp = pd.read_csv('data/disp.csv', delimiter=";")
district = pd.read_csv('data/district.csv', delimiter=";")
loan_test = pd.read_csv('data/loan_test.csv', delimiter=";")
loan_train = pd.read_csv('data/loan_train.csv', delimiter=";")
trans_test = pd.read_csv('data/trans_test.csv', delimiter=";")
trans_train = pd.read_csv('data/trans_train.csv', delimiter=";")

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [104]:
CSV_files = {'account': account, 'card': card_train, 'client': client, 'disp': disp, 'disctrict': district, 'loan': loan_train, 'trans': trans_train}
conn = sqlite3.connect('database.db')

In [40]:
def getCorr(df, size=(11,9)):
    corr = df.corr()
    ax = plt.subplots(figsize=size)
    ax = sns.heatmap(corr,  annot = True, linewidths=.1, mask = np.triu(corr), cmap='coolwarm')

In [13]:
def checkUniqueCategoricalVars():
    for f in CSV_files:
        print("{}:\n {}\n\n".format(f, CSV_files[f].select_dtypes(include=['object']).nunique()))

In [136]:
# Correr só uma vez
for f in CSV_files:
    CSV_files[f].to_sql(f, conn)

ValueError: Table 'account' already exists.

In [132]:
query = """
        SELECT loan_id, loan.account_id, status
        FROM loan, account
        WHERE loan.account_id == account.account_id
        """
         
pd.read_sql_query(query, conn)

,loan_id,account_id,status
0,5314,1787,-1
1,5316,1801,1
2,6863,9188,1
3,5325,1843,1
4,7240,11013,1
...,...,...,...
323,6818,9030,1
324,5625,3189,-1
325,6805,8972,1
326,7233,10963,1


In [134]:
query = """
        SELECT *
        FROM account
        """
         
pd.read_sql_query(query, conn)

DatabaseError: Execution failed on sql '
        SELECT loan_id loan.account_id, status
        FROM loan, account
        WHERE loan.account_id == account.account_id
        ': near ".": syntax error